## Text Summarization Using Tensorflow

---
### Import the necessary libraries

In [1]:
from datasets import load_dataset, load_metric
import transformers
import pandas as pd
from datasets import load_from_disk
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from transformers import AdamWeightDecay

import tensorflow as tf
import numpy as np
import nltk
import warnings
warnings.filterwarnings("ignore")

print(transformers.__version__)

4.25.1


In [2]:
# datasets = raw_datasets.save_to_disk("/home/jvdboss/workspace/Testing_Stuff/summarization_tensorflow/final_data")

---
### Load the data

In [3]:
raw_datasets = load_from_disk("/home/jvdboss/workspace/Testing_Stuff/summarization_tensorflow/final_data")

In [4]:
# Let's check out the data 
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [5]:
def show_random_samples(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_samples(raw_datasets['train'])

,document,summary,id
0,"Father-of-two Darren Mark Harvey, 25, of Portsmouth Road, Woolston, died in the crash in the New Forest in August.\nThe coroner concluded Mr Harvey died as a result of a road traffic collision and it was probable that he steered into the vehicle in a deliberate act.\nHe said he could not be absolutely sure Mr Harvey had intended to kill himself.\nThe crash happened on the A337 close to the Holland Wood Campsite near Brockenhurst late on Saturday 22 August last year.\nMr Harvey's wife, Rebecca, told the inquest that on the night of his death, Mr Harvey had left home in such a distressed state, she had called the police.\nThe inquest was shown footage from a camera mounted on the dashboard of the fire engine that showed Mr Harvey's car swerve into its path.\nHe had a history of depression and was on anti-depressants, the inquest was told.\nMr Harvey's family described him as a ""proud dad, a loving son, brother and grandson who recently married"".","A man who died in a head-on collision with a fire engine probably steered into the vehicle deliberately, an inquest has heard.",35314839
1,"A US Senate investigation said the UK-based bank had been a conduit for ""drug kingpins and rogue nations"".\nMoney laundering is the process of disguising the proceeds of crime so that the money cannot be linked to the wrongdoing.\nHSBC admitted having poor money laundering controls and apologised.\n""We accept responsibility for our past mistakes,"" said HSBC group chief executive Stuart Gulliver in a statement.\n""We have said we are profoundly sorry for them, and we do so again.""\nThe bank said it had spent $290m on improving its systems to prevent money laundering and clawed back some bonuses paid to senior executives in the past.\nBy Robert PestonBusiness editor\nIt also said it expected to reach an agreement with the UK's Financial Services Authority shortly.\nLast month it announced it had set aside $1.5bn to cover the costs of any settlement or fines.\nThe news followed the announcement of a similar but much smaller settlement with UK-based Standard Chartered bank, which will pay $300m in fines for violating US sanctions.\nThe cases are seen as part of a crackdown on money laundering and sanctions violations being led by federal government agencies and New York state authorities.\nThe settlement had been widely expected following a report by the US Senate, published earlier this year, that was heavily critical of HSBC's money laundering controls.\nThe report alleged that:\nThe report suggested HSBC accounts in Mexico and the US were being used by drug barons to launder money.\n""The banks became very overextended, not just in lending on property, which we all know about, but in this case, for example, buying businesses in Mexico about which, it turned out, they knew too little,"" Sir John Gieve, former deputy governor of the Bank of England told the BBC.\nThe Senate report also said HSBC regularly circumvented restrictions on dealings with Iran, North Korea, and other states subject to US sanctions.\nBBC business editor Robert Peston said that as big as the $1.9bn penalty looks, it could have been much worse.\n""HSBC has signed a Deferred Prosecution Agreement for breaches of the US Bank Secrecy Act, the Trading with the Enemy Act and assorted money laundering offences. This is in effect putting the bank on probation,"" he said.\n""But if HSBC had been indicted for these offences, that would have meant that the US government and others could no longer have conducted business with it, which would have been humiliating and highly damaging.""\nThe bank stressed that it had taken on new senior management since the time the problems happened.\nLord Green was chairman of HSBC from 2006 until late 2010 and is now Minister of State for Trade and Investment.\nIn a statement, his department said: ""The report by the US Senate Sub-Committee sets out in detail the evidence submitted to it, and the action taken by HSBC to ens

---
### Preprocessing the data

In [7]:
# Type of Data
type(raw_datasets)

datasets.dataset_dict.DatasetDict

In [8]:
# Loading the metric
metric = load_metric("rouge")

In [9]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [10]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [39]:
# Model checkpoint of pretrained model 
model_checkpoint = "t5-small"

In [12]:
# Applying Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
# Applying Tokenizer on different sentences for checking
tokenizer("Hello, this is a sentence!")

{'input_ids': [8774, 6, 48, 19, 3, 9, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 19, 3, 9, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [15]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this is a sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 19, 3, 9, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [16]:
# If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" 
#(the model can also translate and it needs the prefix to know which task it has to perform).
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We have written the function that will preprocess our samples. We just feed them to the tokenizer with the argument truncation=True. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [17]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
# This function works with one or several examples. In the case of several examples,
# the tokenizer will return a list of lists for each key:
preprocess_function(raw_datasets["train"][:10])

{'input_ids': [[21603, 10, 37, 423, 583, 13, 1783, 16, 20126, 16496, 6, 80, 13, 8, 844, 6025, 4161, 6, 19, 341, 271, 14841, 5, 7057, 161, 19, 4912, 16, 1626, 5981, 11, 186, 7540, 16, 1276, 15, 2296, 7, 5718, 2367, 14621, 4161, 57, 4125, 387, 5, 15059, 7, 30, 8, 4653, 4939, 711, 747, 522, 17879, 788, 12, 1783, 44, 8, 15763, 6029, 1813, 9, 7472, 5, 1404, 1623, 11, 5699, 277, 130, 4161, 57, 18368, 16, 20126, 16496, 227, 8, 2473, 5895, 15, 147, 89, 22411, 139, 8, 1511, 5, 1485, 3271, 3, 21926, 9, 472, 19623, 5251, 8, 616, 12, 15614, 8, 1783, 5, 37, 13818, 10564, 15, 26, 3, 9, 3, 19513, 1481, 6, 18368, 186, 1328, 2605, 30, 7488, 1887, 3, 18, 8, 711, 2309, 9517, 89, 355, 5, 3966, 1954, 9233, 15, 6, 113, 293, 7, 8, 16548, 13363, 106, 14022, 84, 47, 14621, 4161, 6, 243, 255, 228, 59, 7828, 8, 1249, 18, 545, 11298, 1773, 728, 8, 8347, 1560, 5, 611, 6, 255, 243, 72, 1709, 1528, 161, 228, 43, 118, 4006, 91, 12, 766, 8, 3, 19513, 1481, 410, 59, 5124, 5, 96, 196, 17, 19, 1256, 68, 27, 103, 317, 132

To apply this function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [19]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [20]:
# let's take the sample of train, validation and test dataset 
# If you have enough memory and resources you can use the entire data for the project.
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(6000))
small_validation_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

---
Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is sequence-to-sequence (both the input and output are text sequences), we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [21]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

2022-12-14 18:03:13.584861: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 18:03:13.730848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 18:03:13.731013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-14 18:03:13.731923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [22]:
# let's define the cosntants 
batch_size = 1
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-xsum"

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels. Note that our data collators are multi-framework, so make sure you set return_tensors='tf' so you get tf.Tensor objects back and not something else!

We also want to compute ROUGE metrics, which will require us to generate text from our model. To speed things up, we can compile our generation loop with XLA. This results in a huge speedup - up to 100X! The downside of XLA generation, though, is that it doesn't like variable input shapes, because it needs to run a new compilation for each new input shape! To compensate for that, let's use pad_to_multiple_of for the dataset we use for text generation. This will reduce the number of unique input shapes a lot, meaning we can get the benefits of XLA generation with only a few compilations.

In [23]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [24]:
tokenized_datasets["train"]

Dataset({
    features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 204045
})

In [25]:
small_train_dataset

Dataset({
    features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 6000
})

Next, we convert our datasets to tf.data.Dataset, which Keras understands natively. There are two ways to do this - we can use the slightly more low-level Dataset.to_tf_dataset() method, or we can use Model.prepare_tf_dataset(). The main difference between these two is that the Model method can inspect the model to determine which column names it can use as input, which means you don't need to specify them yourself. Make sure to specify the collator we just created as our collate_fn!

In [26]:
train_dataset = model.prepare_tf_dataset(
    small_train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    small_validation_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

test_dataset = model.prepare_tf_dataset(
    small_test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Now we initialize our loss and optimizer and compile the model. Note that most Transformers models compute loss internally - we can train on this as our loss value simply by not specifying a loss when we compile().

In [27]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [28]:
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

---
### Train the data

In [29]:
history = model.fit(
    train_dataset, validation_data=validation_dataset, epochs=1
)

6000/6000 [==============================] - 787s 129ms/step - loss: 2.9422 - val_loss: 2.6128


In [30]:
history.history

{'loss': [2.942176342010498], 'val_loss': [2.612797498703003]}

---
### Evaluate the data

In [31]:
# Let's check the loss of the model 
loss = model.evaluate(test_dataset)
print(loss)

1000/1000 [==============================] - 38s 38ms/step - loss: 2.5948
2.594815731048584


---
### Save the data

In [36]:
# saving the pretrained model
model.save_pretrained("model")

In [40]:
model_name = 'model'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


---
### Taking custom input to check the model performance

In [41]:
document = 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we\'re neglected or forgotten," she said.\n"That may not be true but it is perhaps my perspective over the last few days.\n"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"\nMeanwhile, a flood alert remains in place across the Borders because of the constant rain.\nPeebles was badly hit by problems, sparking calls to introduce more defences in the area.\nScottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.\nThe Labour Party\'s deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.\nHe said it was important to get the flood protection plan right but backed calls to speed up the process.\n"I was quite taken aback by the amount of damage that has been done," he said.\n"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."\nHe said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.\nHave you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.'
if 't5' in model_name: 
    document = "summarize: " + document
tokenized = tokenizer([document], return_tensors='np')
out = model.generate(**tokenized, max_length=128)

Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors


In [42]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

<pad> The damage to the roads in the Scottish Borders has been a "wonderful and aback" for people who have been forced out of their homes, the Labour Party has said.</s>
